In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
df = pd.read_csv('qb.train.csv')

In [3]:
df.head()

,row,body_score,page,answer,text,category,tournaments,answer_type,corr,inlinks
0,1,127.398036,Comus (John Milton),Comus (John Milton),First performed in Ludlow Castle by the childr...,Literature,2000 ACF Nationals,work,True,62
1,2,50.212336,Circe,Comus (John Milton),First performed in Ludlow Castle by the childr...,Literature,2000 ACF Nationals,None,False,5
2,3,44.767071,Satyr,Comus (John Milton),First performed in Ludlow Castle by the childr...,Literature,2000 ACF Nationals,None,False,6
3,4,44.058274,Philip K. Dick,Wilfred Owen,This author is convinced by another to publish...,Literature,2009 ACF Winter,people,False,22
4,5,40.675249,Honore de Balzac,Wilfred Owen,This author is convinced by another to publish...,Literature,2009 ACF Winter,None,False,0


In [6]:
import re
pattern = "^[0-9][0-9][0-9][0-9]"

for index, row in df.iterrows():
    temp = df['tournaments'].values[index]
    year = re.match(pattern, temp)
    df['tournament_year'].values[index] = year.group(0)

In [8]:
for index, row in df.iterrows():
    temp = df['text'].values[index]
    temp = temp.split()
    length = len(temp)
    df['length_text'].values[index] = length

In [9]:
df.head()

,row,body_score,page,answer,text,category,tournaments,answer_type,corr,inlinks,tournament_year,length_text
0,1,127.398036,Comus (John Milton),Comus (John Milton),First performed in Ludlow Castle by the childr...,Literature,2000 ACF Nationals,work,True,62,2000,65
1,2,50.212336,Circe,Comus (John Milton),First performed in Ludlow Castle by the childr...,Literature,2000 ACF Nationals,None,False,5,2000,65
2,3,44.767071,Satyr,Comus (John Milton),First performed in Ludlow Castle by the childr...,Literature,2000 ACF Nationals,None,False,6,2000,65
3,4,44.058274,Philip K. Dick,Wilfred Owen,This author is convinced by another to publish...,Literature,2009 ACF Winter,people,False,22,2009,56
4,5,40.675249,Honore de Balzac,Wilfred Owen,This author is convinced by another to publish...,Literature,2009 ACF Winter,None,False,0,2009,56


In [10]:
X_train = df.iloc[:, [1,6,7,9,10,11]].values
y_train = df.iloc[:, 8].values

In [11]:
label_encoder_tournaments_2 = LabelEncoder()
label_encoder_answer_type_1 = LabelEncoder()
label_encoder_tournament_year_1 = LabelEncoder()

label_encoder_tournaments_2.fit(X_train[:,1]) #never enter multiple column here, it doesn't work
X_train[:,1] = label_encoder_tournaments_2.transform(X_train[:,1])

label_encoder_answer_type_1.fit(X_train[:,2]) #never enter multiple column here, it doesn't work
X_train[:,2] = label_encoder_answer_type_1.transform(X_train[:,2])

label_encoder_tournament_year_1.fit(X_train[:,4]) #never enter multiple column here, it doesn't work
X_train[:,4] = label_encoder_tournament_year_1.transform(X_train[:,4])

one_hot_encoder = OneHotEncoder(categorical_features=[1,2,4], sparse=False)
one_hot_encoder.fit(X_train)
X_train= one_hot_encoder.transform(X_train)

In [12]:
label_encoder_corr = LabelEncoder()
label_encoder_corr.fit(y_train) #never enter multiple column here, it doesn't work
y_train = label_encoder_corr.transform(y_train)

In [13]:
test = pd.read_csv('qb_test.csv')

In [16]:
pattern = "^[0-9][0-9][0-9][0-9]"

for index, row in test.iterrows():
    temp = test['tournaments'].values[index]
    year = re.match(pattern, temp)
    test['tournament_year'].values[index] = year.group(0)

In [18]:
for index, row in test.iterrows():
    temp = test['text'].values[index]
    temp = temp.split()
    length = len(temp)
    test['length_text'].values[index] = length

In [19]:
test.head()

,row,body_score,page,text,category,tournaments,answer_type,inlinks,tournament_year,length_text
0,31,40.023617,Death in Venice,The protagonist of this novella is inspired to...,Literature,2001 ACF Fall,work,4,2001,45
1,32,27.538799,The Eye of the World,The protagonist of this novella is inspired to...,Literature,2001 ACF Fall,work,1,2001,45
2,33,26.976121,Carmilla,The protagonist of this novella is inspired to...,Literature,2001 ACF Fall,work,2,2001,45
3,88,45.848831,A Passage to India,In a symbolic incident in this novel Ronny Hea...,Literature,2001 ACF Regionals,work,5,2001,22
4,91,99.811169,The Cherry Orchard,"We first meet Fiers, the aging footman, in Act...",Literature,2001 ACF Regionals,work,11,2001,96


In [20]:
X_test = test.iloc[:, [1,5,6,7,8,9]].values

In [21]:
X_test[:,1] = label_encoder_tournaments_2.transform(X_test[:,1])

X_test[:,2] = label_encoder_answer_type_1.transform(X_test[:,2])

X_test[:,4] = label_encoder_tournament_year_1.transform(X_test[:,4])

In [22]:
X_test= one_hot_encoder.transform(X_test)

In [23]:
from sklearn.svm import SVC

svm_class = SVC(kernel='rbf', random_state=0, probability=True)
svm_class.fit(X_train,y_train)
y_pred = svm_class.predict(X_test)
y_pred_prob = svm_class.predict_proba(X_test) [:, 1]

In [21]:
#X_test.shape

(887, 88)

In [22]:
#X_train.shape

(8079, 88)

In [29]:
#len(y_pred)

887

In [24]:
final_df = pd.DataFrame(data=y_pred)

In [25]:
final_df.shape

(887, 1)

In [26]:
final_df['row'] = test['row']

In [27]:
final_df.head()

,0,row
0,0,31
1,1,32
2,1,33
3,1,88
4,1,91


In [28]:
final_df.columns = ['corr', 'row']

In [29]:
final_df.head()

,corr,row
0,0,31
1,1,32
2,1,33
3,1,88
4,1,91


In [30]:
final_df.to_csv('prediction_hw3.csv', encoding='utf-8', index=False)